Airbnb NYC 数据集 BT-TWD 可行性实验

本 notebook 按步骤运行：加载配置 → 读取数据 → 预处理 → 桶树划分 → 基线与 BTTWD k 折实验 → 桶级分析。

In [1]:
# 步骤0：环境与路径设置
import os, sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 将项目根目录加入路径，便于导入 bttwdlib
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root_path not in sys.path:
    sys.path.append(root_path)

from bttwdlib import (
    load_yaml_cfg,
    show_cfg,
    load_dataset,
    prepare_features_and_labels,
    BucketTree,
    run_kfold_experiments,
    log_info,
    set_global_seed,
)

cfg_path = Path(root_path) / "configs" / "airbnb_nyc.yaml"
cfg = load_yaml_cfg(cfg_path)
set_global_seed(cfg.get('SEED', {}).get('global_seed', 42))
log_info('【步骤0摘要】环境准备完毕，路径与随机种子已设置。')

【INFO】【2025-12-11 20:16:46】【配置加载】已读取 e:\yan\组\三支决策\机器学习\BT_TWD\configs\airbnb_nyc.yaml
【INFO】【2025-12-11 20:16:49】【步骤0摘要】环境准备完毕，路径与随机种子已设置。


In [2]:
# 步骤1：加载配置
show_cfg(cfg)
log_info('【步骤1摘要】配置文件加载完成，关键参数检查通过。')

【INFO】【2025-12-11 20:16:49】【配置-数据】数据集=airbnb_nyc, k折=5, 目标列=price, 正类="1"
【INFO】【2025-12-11 20:16:49】【配置-BTTWD】阈值模式=None, 全局模型=xgb, 桶内模型=none, 后验估计器(兼容字段)=logreg
【INFO】【2025-12-11 20:16:49】【配置-基线】LogReg启用=True, RandomForest启用=True, KNN启用=True, XGBoost启用=True
【INFO】【2025-12-11 20:16:49】【步骤1摘要】配置文件加载完成，关键参数检查通过。


In [3]:
# 步骤2：加载原始数据
df_raw, target_col_model = load_dataset(cfg)  # 这里返回的是用于建模的标签列，例如 "label"

display(df_raw.head())
print("用于建模的标签列:", target_col_model)

# 1）画 0/1 标签（高价/非高价）的比例
class_counts = df_raw[target_col_model].value_counts(normalize=True)
ax = class_counts.plot(kind='bar', title='高价 vs 非高价比例')
plt.ylabel('比例')

fig_path = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'class_distribution.png')
os.makedirs(os.path.dirname(fig_path), exist_ok=True)
plt.savefig(fig_path, bbox_inches='tight')
plt.close()

# 2）如果想看原始标签列的分布，可以另外单独分析：
raw_target_col = cfg['DATA']['target_col']  # 这里是原始标签列
print("原始目标列:", raw_target_col)
print(df_raw[raw_target_col].describe())

log_info('【步骤2摘要】Airbnb NYC 原始数据加载与基本统计完成。')


【INFO】【2025-12-11 20:16:49】【数据加载】文本表格 ..\data\AB_NYC\AB_NYC_2019.csv 已读取，样本数=48895，列数=16
【INFO】【2025-12-11 20:16:49】【目标变换】已按阈值 255 生成二分类标签列 high_price，正类取 > 255
【INFO】【2025-12-11 20:16:49】【数据集信息】名称=airbnb_nyc，样本数=48895，目标列=high_price，正类比例=10.52%


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,high_price
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365,0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0,0


用于建模的标签列: high_price
原始目标列: price
count    48895.000000
mean       152.720687
std        240.154170
min          0.000000
25%         69.000000
50%        106.000000
75%        175.000000
max      10000.000000
Name: price, dtype: float64
【INFO】【2025-12-11 20:16:49】【步骤2摘要】Airbnb NYC 原始数据加载与基本统计完成。


In [4]:
# 步骤3：预处理与特征工程
X, y, meta = prepare_features_and_labels(df_raw, cfg)
log_info(f'【预处理】编码特征维度={X.shape[1]}，样本数={X.shape[0]}')
log_info(f"【步骤3摘要】特征预处理完成：连续={len(meta['continuous_cols'])}，类别={len(meta['categorical_cols'])}，编码维度={X.shape[1]}。")

【INFO】【2025-12-11 20:16:49】【预处理】缺失值填充策略=most_frequent
【INFO】【2025-12-11 20:16:49】【预处理】连续特征=7个，类别特征=3个
【INFO】【2025-12-11 20:16:49】【预处理】编码后维度=233
【INFO】【2025-12-11 20:16:49】【预处理】编码特征维度=233，样本数=48895
【INFO】【2025-12-11 20:16:49】【步骤3摘要】特征预处理完成：连续=7，类别=3，编码维度=233。


In [5]:
# 步骤4：构建桶树并检查划分
feature_cols_for_bucket = [c for c in df_raw.columns if c != target_col_model]

bucket_tree = BucketTree(
    cfg['BTTWD']['bucket_levels'],
    feature_names=feature_cols_for_bucket
)

bucket_ids_full = bucket_tree.assign_buckets(df_raw[feature_cols_for_bucket])

bucket_df = bucket_ids_full.value_counts().reset_index()
bucket_df.columns = ['bucket_id', 'count']

bucket_df['pos_rate'] = (
    df_raw.groupby(bucket_ids_full)[target_col_model]
    .apply(lambda s: (s == 1).mean())
    .values
)
display(bucket_df.head())
bucket_df.set_index('bucket_id')['count'].plot(kind='bar', figsize=(12,4), title='桶样本数分布')
fig_bucket = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'bucket_metrics_bar.png')
plt.savefig(fig_bucket, bbox_inches='tight')
plt.close()
log_info(f'【步骤4摘要】桶树划分完成，共有 {bucket_ids_full.nunique()} 个叶子桶。')


【INFO】【2025-12-11 20:16:50】【桶树】已为样本生成桶ID，共 45 个组合


,bucket_id,count,pos_rate
0,L1_neighbourhood_group=manhattan|L2_room_type=...,6276,0.100671
1,L1_neighbourhood_group=brooklyn|L2_room_type=p...,5190,0.058824
2,L1_neighbourhood_group=brooklyn|L2_room_type=e...,4857,0.031915
3,L1_neighbourhood_group=manhattan|L2_room_type=...,4311,0.011111
4,L1_neighbourhood_group=manhattan|L2_room_type=...,4020,0.004149


【INFO】【2025-12-11 20:16:51】【步骤4摘要】桶树划分完成，共有 45 个叶子桶。


In [6]:
# 步骤5：运行基线模型 k 折实验
# 基线部分在 run_kfold_experiments 内统一调度
log_info('【步骤5】基线模型将在整体交叉验证中一并运行。')
log_info('【步骤5摘要】基线模型性能将作为后续对比基准。')

【INFO】【2025-12-11 20:16:51】【步骤5】基线模型将在整体交叉验证中一并运行。
【INFO】【2025-12-11 20:16:51】【步骤5摘要】基线模型性能将作为后续对比基准。


In [7]:
import numpy as np
import pandas as pd

print("y 全局标签分布：", np.unique(y, return_counts=True))

print("原始标签列分布：")
print(df_raw[cfg['DATA']['target_col']].value_counts())


y 全局标签分布： (array([0, 1]), array([43751,  5144], dtype=int64))
原始标签列分布：
price
100    2051
150    2047
50     1534
60     1458
200    1401
       ... 
780       1
386       1
888       1
483       1
338       1
Name: count, Length: 674, dtype: int64


In [8]:
# 步骤6：运行 BTTWD k 折实验（含基线）
results = run_kfold_experiments(X, y, df_raw.drop(columns=[cfg['DATA']['target_col']]), cfg)
summary_df = pd.read_csv(os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'metrics_kfold_summary.csv'))
display(summary_df)
summary_df.plot(x='model', kind='bar', figsize=(8,4), title='模型指标对比')
fig_compare = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'metrics_compare.png')
plt.savefig(fig_compare, bbox_inches='tight')
plt.close()
log_info('【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。')

【INFO】【2025-12-11 20:16:51】【基线-LogReg】使用决策阈值=0.300（fixed 模式）


e:\yan\组\三支决策\机器学习\BT_TWD\bttwdlib\baselines.py:103: RuntimeWarning: Mean of empty slice
  summary[f"{col}_mean"] = float(np.nanmean(arr))
d:\Anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


【INFO】【2025-12-11 20:17:11】【基线-LogReg】整体指标：AUC_mean=0.839, AUC_std=0.007, BAC_mean=0.648, BAC_std=0.009, BND_ratio_mean=0.000, BND_ratio_std=0.000, F1_mean=0.381, F1_std=0.016, Kappa_mean=0.313, Kappa_std=0.017, MCC_mean=0.314, MCC_std=0.017, POS_Coverage_mean=nan, POS_Coverage_std=nan, Precision_mean=0.407, Precision_std=0.015, Recall_mean=0.358, Recall_std=0.017, Regret_mean=0.325, Regret_std=0.008
【INFO】【2025-12-11 20:17:11】【基线-RF】使用决策阈值=0.300（fixed 模式）


e:\yan\组\三支决策\机器学习\BT_TWD\bttwdlib\baselines.py:103: RuntimeWarning: Mean of empty slice
  summary[f"{col}_mean"] = float(np.nanmean(arr))
d:\Anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


【INFO】【2025-12-11 20:17:35】【基线-RF】整体指标：AUC_mean=0.864, AUC_std=0.006, BAC_mean=0.717, BAC_std=0.010, BND_ratio_mean=0.000, BND_ratio_std=0.000, F1_mean=0.486, F1_std=0.015, Kappa_mean=0.423, Kappa_std=0.017, MCC_mean=0.423, MCC_std=0.017, POS_Coverage_mean=nan, POS_Coverage_std=nan, Precision_mean=0.471, Precision_std=0.015, Recall_mean=0.501, Recall_std=0.019, Regret_mean=0.269, Regret_std=0.008
【INFO】【2025-12-11 20:17:35】【基线-KNN】使用决策阈值=0.300（fixed 模式）


...

d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classif

【INFO】【2025-12-11 20:19:02】[BASELINE] 桶 L1_neighbourhood_group=manhattan|L2_room_type=entire_home|L3_availability_365=rare: BAC=0.6269, Regret=0.5008, Precision=0.4061, Recall=0.3490
【INFO】【2025-12-11 20:19:02】[BASELINE] 桶 L1_neighbourhood_group=brooklyn|L2_room_type=shared_room|L3_availability_365=long_term: BAC=0.5000, Regret=0.1026, Precision=0.0000, Recall=0.0000
【INFO】【2025-12-11 20:19:02】[BASELINE] 桶 L1_neighbourhood_group=manhattan|L2_room_type=private_room|L3_availability_365=long_term: BAC=0.7178, Regret=0.2295, Precision=0.7241, Recall=0.4565
【INFO】【2025-12-11 20:19:02】[BASELINE] 桶 L1_neighbourhood_group=brooklyn|L2_room_type=entire_home|L3_availability_365=mid_term: BAC=0.5711, Regret=0.4467, Precision=0.3824, Recall=0.1970
【INFO】【2025-12-11 20:19:02】[BASELINE] 桶 L1_neighbourhood_group=queens|L2_room_type=private_room|L3_availability_365=rare: BAC=0.5000, Regret=0.0385, Precision=0.0000, Recall=0.0000
【INFO】【2025-12-11 20:19:02】[BASELINE] 桶 L1_neighbourhood_group=queens|L2_r

d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classif

,model,Precision_mean,Precision_std,Recall_mean,Recall_std,F1_mean,F1_std,BAC_mean,BAC_std,AUC_mean,...,MCC_mean,MCC_std,Kappa_mean,Kappa_std,BND_ratio_mean,BND_ratio_std,POS_Coverage_mean,POS_Coverage_std,Regret_mean,Regret_std
0,BTTWD,0.384840,0.006776,0.642879,0.020605,0.481404,0.009923,0.761030,0.009452,0.872600,...,0.420836,0.012402,0.402819,0.010957,0.125841,0.01847,0.120667,0.020474,0.264291,0.007819
1,LogReg,0.407228,0.014730,0.358081,0.017291,0.381024,0.015853,0.648412,0.008821,0.839357,...,0.314289,0.016890,0.313467,0.017008,0.000000,0.00000,NaN,NaN,0.324941,0.007575
2,RandomForest,0.471363,0.014891,0.500965,0.019301,0.485608,0.015446,0.717443,0.009743,0.863615,...,0.423385,0.017111,0.423080,0.017038,0.000000,0.00000,NaN,NaN,0.269128,0.008426
3,KNN,0.351642,0.005035,0.535766,0.011975,0.424559,0.006247,0.709804,0.005331,0.785738,...,0.350912,0.007413,0.340831,0.006891,0.000000,0.00000,NaN,NaN,0.299294,0.004099
4,XGBoost,0.485927,0.007411,0.485221,0.012926,0.485541,0.009800,0.712440,0.006401,0.875833,...,0.425137,0.010500,0.425119,0.010517,0.000000,0.00000,NaN,NaN,0.270621,0.005297


【INFO】【2025-12-11 20:19:03】【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。


In [9]:
# 步骤7：桶级别分析
bucket_metrics_path = os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'bucket_metrics.csv')
if os.path.exists(bucket_metrics_path):
    bucket_metrics_df = pd.read_csv(bucket_metrics_path)
    display(bucket_metrics_df.head())
    bucket_metrics_df.plot(x='bucket_id', y='pos_rate_all', kind='bar', figsize=(12,4), title='桶正类比例')
    plt.ylabel('正类比例')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(fig_bucket, bbox_inches='tight')
    plt.close()
log_info('【步骤7摘要】桶级指标已整理，可用于局部化分析。')

,bucket_id,layer,parent_bucket_id,n_train,n_val,pos_rate_train,pos_rate_val,alpha,beta,regret_val,...,is_weak,threshold_source_bucket,parent_with_threshold,n_test,pos_rate_test,BND_ratio_test,POS_Coverage_test,regret_test,fold,pos_rate
0,ROOT,L1,NaN,5474,2345,0.105955,0.096802,0.4,0.2,0.259701,...,False,ROOT,NaN,NaN,NaN,NaN,NaN,NaN,1,0.105225
1,L1_neighbourhood_group=manhattan,L1,ROOT,2435,1036,0.173717,0.172780,0.4,0.2,0.373552,...,True,ROOT,NaN,NaN,NaN,NaN,NaN,NaN,1,0.171109
2,L1_neighbourhood_group=brooklyn,L1,ROOT,2232,967,0.071237,0.073423,0.2,0.1,0.197518,...,False,L1_neighbourhood_group=brooklyn,NaN,NaN,NaN,NaN,NaN,NaN,1,0.059095
3,L1_neighbourhood_group=manhattan|L2_room_type=...,L2,L1_neighbourhood_group=manhattan,1481,636,0.245105,0.272013,0.4,0.2,0.526730,...,True,ROOT,NaN,NaN,NaN,NaN,NaN,NaN,1,0.253068
4,L1_neighbourhood_group=brooklyn|L2_room_type=p...,L2,L1_neighbourhood_group=brooklyn,1124,488,0.006228,0.008197,0.2,0.1,0.032787,...,True,L1_neighbourhood_group=brooklyn,L1_neighbourhood_group=brooklyn,NaN,NaN,NaN,NaN,NaN,1,0.008796


C:\Users\xzq20.HUANLI\AppData\Local\Temp\ipykernel_18312\946468196.py:9: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


【INFO】【2025-12-11 20:19:06】【步骤7摘要】桶级指标已整理，可用于局部化分析。


In [10]:
# 步骤8：结果汇总
log_info('【步骤8】检查结果文件与图表。')
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['results_dir'])))
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['figs_dir'])))
log_info('【全部步骤完成】Airbnb NYC 数据集上的 BT-TWD 实验结束。')

【INFO】【2025-12-11 20:19:06】【步骤8】检查结果文件与图表。
['baseline_bucket_metrics.csv', 'bucket_fallback_stats.csv', 'bucket_metrics.csv', 'bucket_metrics_gain.csv', 'bucket_thresholds.csv', 'bucket_thresholds_per_fold.csv', 'bucket_tree_structure.csv', 'metrics_kfold_per_fold.csv', 'metrics_kfold_summary.csv', 'metrics_overview.csv']
['bank_class_distribution.png', 'bucket_metrics_bar.png', 'class_distribution.png', 'metrics_compare.png']
【INFO】【2025-12-11 20:19:06】【全部步骤完成】Airbnb NYC 数据集上的 BT-TWD 实验结束。
